In [45]:
import gutenbergpy.textget
from gutenbergpy.gutenbergcache import GutenbergCache
import numpy as np

In [2]:
cache = None
try:
    cache = GutenbergCache.get_cache()
except:
    GutenbergCache.create()
    cache = GutenbergCache.get_cache()

In [50]:
authors_list = cache.native_query("""
SELECT author.name
FROM books book
JOIN book_authors book_author ON book_author.bookid = book.id
JOIN authors author ON author.id = book_author.authorid
""").fetchall()
for author in set(np.array(authors_list).flatten()):
    if "Tolstoy" in author:
        print(author)

Tolstoy van Aylde-Jonghe, Elzélina
Tolstoy, Lev N.
Tolstoy, Leo, graf
Tolstoy, Alexandra, Countess
Tolstoy, Sonya
Tolstoy, A. K. (Aleksey Konstantinovich), graf
Tolstoy, Sophie Andreevna
Tolstoy, Aleksey Konstantinovich, graf
Tolstoy, Aleksei Konstantinovich, Count
Tolstoy, Ilya, Count
Tolstoy, Alexis Konstantinovich, graf
Tolstoy, Alexei Konstantinovich, graf
Tolstoy, Graf Leo


In [62]:
cache.query(authors=['Tolstoy, Graf Leo'])

[46086,
 50172,
 43761,
 43302,
 38690,
 47197,
 43794,
 13214,
 28920,
 6157,
 17951,
 44266,
 52350,
 51573,
 17950,
 67224,
 689,
 17552,
 243,
 41119,
 49522,
 26472,
 51708,
 26665,
 26662,
 15628,
 27726,
 53201,
 52981,
 26663,
 26664,
 50482,
 52774,
 2142,
 71817,
 4761,
 54560,
 64908,
 43372,
 49487,
 53721,
 985,
 34687,
 47720,
 49824,
 69861,
 45297,
 46272,
 71636,
 46447,
 45298,
 60579,
 67556,
 61388,
 38915,
 56399,
 17352,
 53470,
 58858,
 56797,
 4602,
 12604,
 43409,
 1399,
 3630,
 17949,
 27189,
 19681,
 26661,
 26666,
 19680,
 9792,
 3631,
 59195,
 52982,
 1938,
 67250,
 26660,
 2600,
 44957,
 45271,
 13259,
 51018,
 7176,
 2450,
 3540,
 2637,
 12379,
 45284,
 44956,
 37008,
 38616,
 26581,
 3541,
 33266,
 38025,
 986]